# Exportation et Visualisation de Données GEE

## Authentification et Initialisation de Google Earth Engine
Dans cette section, nous authentifions et initialisons Google Earth Engine (GEE) pour accéder aux données géospatiales.

## Création d'une Carte Interactive
Nous créons une carte interactive avec des outils de dessin pour définir une région d'intérêt (ROI).

## Téléchargement des Données SRTM
Après avoir défini la ROI, nous téléchargeons les données SRTM (Shuttle Radar Topography Mission) pour cette région et les sauvegardons localement.

## Exportation des Images Sentinel-2 vers Google Drive
Nous exportons les images Sentinel-2 pour la région définie vers Google Drive, en spécifiant une plage de dates et en filtrant les images avec moins de 20% de couverture nuageuse.

In [1]:
import geemap
import ee

# Authentifier et initialiser Google Earth Engine
geemap.ee_initialize()

# Créer une carte interactive
m = geemap.Map(center=[0, 0], zoom=2)

# Ajouter des outils de dessin pour dessiner une zone d'intérêt (ROI)
m.add_basemap("SATELLITE")  # Choisir un fond de carte satellite
m.add_draw_control()  # Activer l'outil de dessin
m

# Une fois la zone d'intérêt dessinée sur la carte, vous pouvez obtenir la géométrie


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [5]:
# Une fois que vous avez dessiné la région d'intérêt (ROI), récupérez la géométrie
region = m.user_roi  # m.user_roi contient la géométrie dessinée

# Vérifiez que la géométrie a bien été capturée
if region is None:
    print("Veuillez dessiner une région d'intérêt avant de continuer.")
else:
    # Charger les données SRTM depuis GEE
    srtm = ee.Image("USGS/SRTMGL1_003")

    # Chemin où vous voulez sauvegarder l'image
    out_dir = r"D:\UQTR\Automne 2024\Ouest\SRTM"
    out_file = 'SRTM_DEM_test.tif'

    # Exporter l'image SRTM pour la région dessinée
    geemap.ee_export_image(
        srtm, 
        filename=f"{out_dir}/{out_file}", 
        scale=30, 
        region=region, 
        file_per_band=False
    )

    print(f"Fichier DEM téléchargé et sauvegardé sous : {out_dir}/{out_file}")


Tâche d'exportation vers Google Drive démarrée. Veuillez vérifier votre Google Drive.
Tâche d'exportation vers Google Drive démarrée. Veuillez vérifier votre Google Drive.


## Sentinel 2 eport to Google Drive

In [8]:
# Une fois que vous avez dessiné la région d'intérêt (ROI), récupérez la géométrie
region = m.user_roi  # m.user_roi contient la géométrie dessinée

# Vérifiez que la géométrie a bien été capturée
if region is None:
    print("Veuillez dessiner une région d'intérêt avant de continuer.")
else:
    # Définir la plage de dates pour les images Sentinel-2
    start_date = '2023-5-01'
    end_date = '2023-7-31'

    # Charger les images Sentinel-2 pour la région et la plage de dates spécifiées
    sentinel2_collection = ee.ImageCollection('COPERNICUS/S2') \
        .filterDate(start_date, end_date) \
        .filterBounds(region) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .sort('CLOUDY_PIXEL_PERCENTAGE') \
        .first()

    # Exporter l'image Sentinel-2 pour la région dessinée vers Google Drive
    task = ee.batch.Export.image.toDrive(
        image=sentinel2_collection,
        description='Sentinel2_test',
        folder='EarthEngineImages',
        fileNamePrefix='Sentinel2_test',
        region=region,
        scale=10,
        fileFormat='GeoTIFF'
    )
    task.start()

    print("Tâche d'exportation vers Google Drive démarrée. Veuillez vérifier votre Google Drive.")

EEException: ImageCollection.load: ImageCollection asset 'LANDSAT/LC08/C01/T1_SR' not found (does not exist or caller does not have access).

: 